ETHZ - 2022 \\

AI4Good - ReforesTree project \\

Klim Troyan & Silviu Nastasescu & Dominic Wong

## **Requirements**

In [ ]:
# The requirements.txt file should be in the working directory
# The paths for the field dataset and images dataset (i.e, the tiles) should be changed according to the local setting. 

# The matplotlib library seems to need to be installed after the requirements.txt file

# **Install dependencies**

In [ ]:
%pip install -r requirements.txt
%pip install torchgeo
%pip install -U matplotlib
%pip install wandb --upgrade -Uqqq

In [ ]:
import os

# import torch and neural networks libraries needed
import torch
import torch.nn as nn
import torch.nn.functional as F   # for useful functions (e.g. activation)
import torch.optim
import torchvision
from torchvision import transforms


# import the sklearn metrics needed
import sklearn
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score, r2_score, mean_squared_error

# numerical operations without torch
import numpy as np
import random

# import pandas to read the files
import pandas as pd

# visualizations
from matplotlib import pyplot as plt

# to copy e.g. the model from gpu to cpu, etc.
import copy

# to split the training data into train/val and to get data mini-batches
from torch.utils.data import Dataset, DataLoader, random_split

# tracking of experiments
from tqdm import trange, tqdm
import wandb



# Fix seeds for reproducibility
SEED = 121997
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = False

# **Data**

### **Load datasets from files**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# dataset_field_data_path = "/content/drive/MyDrive/ETHZ/AI4Good/final_dataset.csv"
# dataset_tiles_path = "/content/drive/MyDrive/ETHZ/AI4Good/tiles/"

dataset_field_data_path = "./data/mapping/final_dataset.csv"
dataset_tiles_path = "./data/tiles/"

In [ ]:
dataset_field = pd.read_csv(dataset_field_data_path)
dataset_field

In [ ]:
# Show all the column names in the dataset
print(dataset_field.columns.to_list())

In [ ]:
# Drop unnecessary columns
columns_to_drop = ['img_name', 'year',
       'tile_index', 'tile_xmin', 'tile_ymin', 'tile_xmax', 'tile_ymax', 'x',
       'y', 'Xmin', 'Ymin', 'Xmax', 'Ymax', 'X_d', 'Y_d',
       'is_musacea_d', 'plot_id', 'tree_id', 'is_musacea_g',
       'X_g', 'Y_g', 'group', 'id']

dataset_field = dataset_field.drop(columns=columns_to_drop, axis=1)

In [ ]:
# Rename the sites to avoid having spaces in the values
dataset_field["site"] = dataset_field["site"].apply(lambda x: x.lower())
dataset_field["site"] = dataset_field["site"].apply(lambda x: x.replace(" ", "_"))

In [ ]:
# Show how a sample looks like
example_sample = dataset_field.iloc[0]  # choose the first sample
print(example_sample)

In [ ]:
# Print all the different sites' names
print(dataset_field["site"].unique())

### Experiment with different pre-processing for the preparation of the sampled dataset

In [ ]:
# Experiment with some pre-processing for "extreme" settings

# banana = dataset_field[dataset_field["name"] == "Musacea"]
# banana_mid_point = len(banana) // 2
# banana_agb = np.sort(banana['AGB'])
# banana = banana[(banana['AGB'] >= banana_agb[banana_mid_point - 300]) & (banana['AGB'] <= banana_agb[banana_mid_point + 300])].reset_index(drop=True).iloc[:600]

# non_banana = dataset_field.loc[dataset_field["name"] == "Cacao"]
# non_banana_mid_point = len(non_banana) // 2
# non_banana_agb = np.sort(non_banana['AGB'])
# non_banana = non_banana[(non_banana['AGB'] >= non_banana_agb[non_banana_mid_point - 300]) & (non_banana['AGB'] <= non_banana_agb[non_banana_mid_point + 300])].reset_index(drop=True).iloc[:600]

# sampled_dataset = pd.concat([banana, non_banana])
# sampled_dataset = sampled_dataset.iloc[np.random.permutation(len(sampled_dataset))].reset_index(drop=True)
# sampled_dataset

# ---

# banana = dataset_field[dataset_field["is_banana"] == True]
# banana_agb_max = np.sort(banana['AGB'])[600]
# banana = banana[banana['AGB'] <= banana_agb_max].reset_index(drop=True).iloc[:600]

# non_banana = dataset_field.loc[dataset_field["is_banana"] == False]
# non_banana_agb_max = np.sort(non_banana['AGB'])[600]
# non_banana = non_banana[non_banana['AGB'] <= non_banana_agb_max].reset_index(drop=True).iloc[:600]

# sampled_dataset = pd.concat([banana, non_banana])
# sampled_dataset = sampled_dataset.iloc[np.random.permutation(len(sampled_dataset))].reset_index(drop=True)
# sampled_dataset

In [ ]:
# # Removing the first and last nr_outliers_at_end
# nr_outliers_at_end = 200
# min_val, max_val = np.sort(dataset_field['AGB'])[[nr_outliers_at_end, -nr_outliers_at_end]]
# dataset_field = dataset_field[(dataset_field['AGB'] > min_val) & (dataset_field['AGB'] < max_val)].reset_index(drop=True)
# print(f'{min_val=}, {max_val=}')
# dataset_field

### Actual paper's pre-processing for the preparation of the sampled dataset

In [ ]:
# Create subsets of the dataset (with full union) per site
carlos_vera_guevara_site_dataset = dataset_field.loc[dataset_field["site"] == "carlos_vera_guevara_rgb"]
carlos_vera_arteaga_site_dataset = dataset_field.loc[dataset_field["site"] == "carlos_vera_arteaga_rgb"]
flora_pluas_site_dataset = dataset_field.loc[dataset_field["site"] == "flora_pluas_rgb"]
leonor_aspiazu_site_dataset = dataset_field.loc[dataset_field["site"] == "leonor_aspiazu_rgb"]
manuel_macias_site_dataset = dataset_field.loc[dataset_field["site"] == "manuel_macias_rgb"]
nestor_macias_site_dataset = dataset_field.loc[dataset_field["site"] == "nestor_macias_rgb"]

print(carlos_vera_guevara_site_dataset.shape)
print(carlos_vera_arteaga_site_dataset.shape)
print(flora_pluas_site_dataset.shape)
print(leonor_aspiazu_site_dataset.shape)
print(manuel_macias_site_dataset.shape)
print(nestor_macias_site_dataset.shape)

In [ ]:
# Create two subsets per site for banana/non-banana from the site subsets
banana_carlos_vera_guevara_site_dataset = carlos_vera_guevara_site_dataset.loc[carlos_vera_guevara_site_dataset["is_banana"] == True]
non_banana_carlos_vera_guevara_site_dataset = carlos_vera_guevara_site_dataset.loc[carlos_vera_guevara_site_dataset["is_banana"] == False]

banana_carlos_vera_arteaga_site_dataset = carlos_vera_guevara_site_dataset.loc[carlos_vera_guevara_site_dataset["is_banana"] == True]
non_banana_carlos_vera_arteaga_site_dataset = carlos_vera_guevara_site_dataset.loc[carlos_vera_guevara_site_dataset["is_banana"] == False]

banana_flora_pluas_site_dataset = flora_pluas_site_dataset.loc[flora_pluas_site_dataset["is_banana"] == True]
non_banana_flora_pluas_site_dataset = flora_pluas_site_dataset.loc[flora_pluas_site_dataset["is_banana"] == False]

banana_leonor_aspiazu_site_dataset = leonor_aspiazu_site_dataset.loc[leonor_aspiazu_site_dataset["is_banana"] == True]
non_banana_leonor_aspiazu_site_dataset = leonor_aspiazu_site_dataset.loc[leonor_aspiazu_site_dataset["is_banana"] == False]

banana_manuel_macias_site_dataset = manuel_macias_site_dataset.loc[manuel_macias_site_dataset["is_banana"] == True]
non_banana_manuel_macias_site_dataset = manuel_macias_site_dataset.loc[manuel_macias_site_dataset["is_banana"] == False]

banana_nestor_macias_site_dataset = nestor_macias_site_dataset.loc[nestor_macias_site_dataset["is_banana"] == True]
non_banana_nestor_macias_site_dataset = nestor_macias_site_dataset.loc[nestor_macias_site_dataset["is_banana"] == False]

In [ ]:
print(banana_carlos_vera_guevara_site_dataset.shape)
print(non_banana_carlos_vera_guevara_site_dataset.shape)

print(banana_carlos_vera_arteaga_site_dataset.shape)
print(non_banana_carlos_vera_arteaga_site_dataset.shape)

print(banana_flora_pluas_site_dataset.shape)
print(non_banana_flora_pluas_site_dataset.shape)

print(banana_leonor_aspiazu_site_dataset.shape)
print(non_banana_leonor_aspiazu_site_dataset.shape)

print(banana_manuel_macias_site_dataset.shape)
print(non_banana_manuel_macias_site_dataset.shape)

print(banana_nestor_macias_site_dataset.shape)
print(non_banana_nestor_macias_site_dataset.shape)

In [ ]:
# We can sample 100 data samples from each of the subsets (pandas dataframes) created

sampled_df1 = banana_carlos_vera_guevara_site_dataset.sample(n=100)
sampled_df2 = non_banana_carlos_vera_guevara_site_dataset.sample(n=100)
sampled_df3 = banana_carlos_vera_arteaga_site_dataset.sample(n=100)
sampled_df4 = non_banana_carlos_vera_arteaga_site_dataset.sample(n=100)
sampled_df5 = banana_flora_pluas_site_dataset.sample(n=100)
sampled_df6 = non_banana_flora_pluas_site_dataset.sample(n=100)
sampled_df7 = banana_leonor_aspiazu_site_dataset.sample(n=100)
sampled_df8 = non_banana_leonor_aspiazu_site_dataset.sample(n=100)
sampled_df9 = banana_manuel_macias_site_dataset.sample(n=100)
sampled_df10 = non_banana_manuel_macias_site_dataset.sample(n=100)
sampled_df11 = banana_nestor_macias_site_dataset.sample(n=100)
sampled_df12 = non_banana_nestor_macias_site_dataset.sample(n=100)

print(sampled_df1.shape)
print(sampled_df2.shape)
print(sampled_df3.shape)
print(sampled_df4.shape)
print(sampled_df5.shape)
print(sampled_df6.shape)
print(sampled_df7.shape)
print(sampled_df8.shape)
print(sampled_df9.shape)
print(sampled_df10.shape)
print(sampled_df11.shape)
print(sampled_df12.shape)

In [ ]:
# Create the fully sampled dataset for our experiment (1200 trees as per the paper)
sampled_dataset = pd.concat([sampled_df1, sampled_df2, sampled_df3, sampled_df4, sampled_df5, sampled_df6, sampled_df7, sampled_df8, sampled_df9, sampled_df10, sampled_df11, sampled_df12])
sampled_dataset = sampled_dataset.iloc[np.random.permutation(len(sampled_dataset))]

In [ ]:
# Create the train-val-test datasets from the above dataset by sampling randomly
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(sampled_dataset, test_size=0.3, random_state=SEED)
train_df, test_df = train_test_split(train_df, test_size=0.1, random_state=SEED)

print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

In [ ]:
print(test_df)

### **Extract the individual trees from the bounding boxes & Prepare the train-val-test lists of data (i.e., image trees, agb values, tree group labels)**

In [ ]:
nn_input_image_size = 800   # choice from the ReforesTree paper to make sure to match bigger tree images too

def bbox_float_to_int(bbox):
    return [round(coordinate) for coordinate in bbox]

def reshape_bbox(tree_image):
    W_padding = nn_input_image_size - tree_image.shape[2]   # width padding  
    H_padding = nn_input_image_size - tree_image.shape[1]   # height padding

    left_W_padding = W_padding // 2
    right_W_padding = W_padding // 2
    top_H_padding = H_padding // 2
    bottom_H_padding = H_padding // 2

    if tree_image.shape[2] + left_W_padding + right_W_padding == nn_input_image_size - 1:
        right_W_padding = right_W_padding + 1
    if tree_image.shape[1] + top_H_padding + bottom_H_padding == nn_input_image_size - 1:
        bottom_H_padding = bottom_H_padding + 1
    
    # See: https://pytorch.org/docs/stable/generated/torch.nn.functional.pad.html
    p2d = (left_W_padding, right_W_padding, top_H_padding, bottom_H_padding)  # pad the left side and right side each by 10, pad the upper side and lower side each by 20
    
    padded_tree_image = F.pad(tree_image, p2d, "constant", 0)

    return padded_tree_image

from PIL import Image
from torchvision import transforms

def get_tree_from_image(sample):
    image_path_i = sample['img_path']
    index = image_path_i.find("_")
    site_folder_name = image_path_i[:index]
    image_path_i = dataset_tiles_path + site_folder_name + "/" + sample['img_path']     # create the full path to get the image
    
    bbox = [sample['xmin'], sample['ymin'], sample['xmax'], sample['ymax']]     # store the four components of the bbox in a list 
    # print(bbox)
    _num_channels = 3   # RGB image is considered

    
    # print("Getting the tree at image file location: ", image_path_i)
    image = Image.open(image_path_i)
    to_tensor = transforms.ToTensor()
    image = to_tensor(image)    # make the image a Tensor

    bbox = bbox_float_to_int(bbox)    # cast coordinates of bbox from float to int    
    
    tree_binary_label = sample['is_banana']    # i.e., the binary group banana/non-banana
    tree_agb_value = sample['AGB']

    tree_image = image[:, int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]     # crop the image to bounding box
    # plt.imshow((tree_image).permute(1, 2, 0))

    tree_image_reshaped = reshape_bbox(tree_image)      # fit the cropped image to shape of 800x800

    tree_image_reshaped_normalized = tree_image_reshaped    # normalize the pixel values    ??? We do NOT normalize??? We get black pixels when we divide by 255...

    return tree_image_reshaped_normalized.to(torch.float), tree_binary_label, tree_agb_value.astype(dtype=np.float32)


In [ ]:
## Sample the trees from the image samples

def sample_trees(df, df_mode, set_nb_of_samples=None):

    tree_images = []
    tree_labels = []
    tree_agb_values = []

    total_number_of_trees = 0

    if set_nb_of_samples is None:
        set_nb_of_samples = df.shape[0]

    print(f"Total number of tree samples for {df_mode}: ", set_nb_of_samples)

    for i in range(set_nb_of_samples):
        sample = df.iloc[i]        
        tree_i, label_i, agb_value_i = get_tree_from_image(sample)
        tree_images.append(tree_i)
        tree_labels.append(label_i)
        tree_agb_values.append(agb_value_i)

    return tree_images, tree_labels, tree_agb_values


# /!\ Do not forget to set set_nb_of_samples=None if we want to use the full sampled dataset
train_tree_images, train_tree_labels, train_tree_agb_values = sample_trees(train_df, df_mode="train", set_nb_of_samples=None)
print("Done with getting the trees for training")
val_tree_images, val_tree_labels, val_tree_agb_values = sample_trees(val_df, df_mode="val", set_nb_of_samples=None)
print("Done with getting the trees for validation")
test_tree_images, test_tree_labels, test_tree_agb_values = sample_trees(test_df, df_mode="test", set_nb_of_samples=None)
print("Done with getting the trees for testing")

In [ ]:
# print(train_tree_images)
# print(train_tree_labels)
# print(train_tree_agb_values)

### **Check the state of our data until here**

In [ ]:
# Print the results of our pre-processing to get lists with all the samples needed
print(train_tree_images)
print(train_tree_labels)
print(train_tree_agb_values)
print("\n")

In [ ]:
print(f"List of TRAIN {len(train_tree_images)} tree samples represented as tensors of shape {train_tree_images[0].shape}")
print("Total number of binary labels (i.e., banana/non-banana): ", len(train_tree_labels))
print("Total number of AGB values: ", len(train_tree_agb_values))

print(f"List of VAL {len(val_tree_images)} tree samples represented as tensors of shape {val_tree_images[0].shape}")
print("Total number of binary labels (i.e., banana/non-banana): ", len(val_tree_labels))
print("Total number of AGB values: ", len(val_tree_agb_values))

print(f"List of TEST {len(test_tree_images)} tree samples represented as tensors of shape {test_tree_images[0].shape}")
print("Total number of binary labels (i.e., banana/non-banana): ", len(test_tree_labels))
print("Total number of AGB values: ", len(test_tree_agb_values))

### **One-Hot Encode (OHE) the tree group labels**

In [ ]:
## OHE the tree group label values

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


## Training
values = array(train_tree_labels)
train_label_encoder = LabelEncoder()
integer_encoded = train_label_encoder.fit_transform(values)
train_onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
train_onehot_encoded = train_onehot_encoder.fit_transform(integer_encoded)
# inverted = train_label_encoder.inverse_transform([argmax(train_onehot_encoded[0, :])])
# print(inverted)

train_labels_list = []
for i in range(train_onehot_encoded.shape[0]):
    train_labels_list.append(torch.Tensor(train_onehot_encoded[i, :]))

print("One Hot Encoded (OHE) tree group labels for TRAINING: \n", train_labels_list)


## Validation
values = array(val_tree_labels)
val_label_encoder = LabelEncoder()
integer_encoded = val_label_encoder.fit_transform(values)
val_onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
val_onehot_encoded = val_onehot_encoder.fit_transform(integer_encoded)
# inverted = val_label_encoder.inverse_transform([argmax(val_onehot_encoded[0, :])])
# print(inverted)

val_labels_list = []
for i in range(val_onehot_encoded.shape[0]):
    val_labels_list.append(torch.Tensor(val_onehot_encoded[i, :]))

print("One Hot Encoded (OHE) tree group labels for VALIDATION: \n", val_labels_list)


## Testing
values = array(test_tree_labels)
test_label_encoder = LabelEncoder()
integer_encoded = test_label_encoder.fit_transform(values)
test_onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
test_onehot_encoded = test_onehot_encoder.fit_transform(integer_encoded)
# inverted = test_label_encoder.inverse_transform([argmax(test_onehot_encoded[0, :])])
# print(inverted)

test_labels_list = []
for i in range(test_onehot_encoded.shape[0]):
    test_labels_list.append(torch.Tensor(test_onehot_encoded[i, :]))

# print("One Hot Encoded (OHE) tree group labels for TESTING: \n", test_labels_list)

### **See what are the unique values taken by our feature(s) and target**

In [ ]:
## Count how many trees per binary group of trees we have in our training set, validation set and testing set.
## It should be quite balanced.
# Note: 
# element 0 is non-banana
# element 1 is banana

from collections import Counter

## For TRAINING
print("\n", train_labels_list)
train_tree_labels_list = [ohe_label for ohe_label in train_labels_list]

# inverted = train_label_encoder.inverse_transform([argmax(train_onehot_encoded[0, :])]).tolist()
# print(inverted)

train_original_labels = [train_label_encoder.inverse_transform([argmax(train_onehot_encoded[i, :])]).tolist()[0] for i in range(len(train_tree_labels_list))]
print(train_original_labels)
print(Counter(train_original_labels).keys())
print(Counter(train_original_labels).values())


## For VALIDATION
print("\n", val_labels_list)
val_tree_labels_list = [ohe_label for ohe_label in val_labels_list]

# inverted = val_label_encoder.inverse_transform([argmax(val_onehot_encoded[0, :])]).tolist()
# print(inverted)

val_original_labels = [val_label_encoder.inverse_transform([argmax(val_onehot_encoded[i, :])]).tolist()[0] for i in range(len(val_tree_labels_list))]
print(val_original_labels)
print(Counter(val_original_labels).keys())
print(Counter(val_original_labels).values())

## For TESTING
print("\n", test_labels_list)
test_tree_labels_list = [ohe_label for ohe_label in test_labels_list]

# inverted = test_label_encoder.inverse_transform([argmax(test_onehot_encoded[0, :])]).tolist()
# print(inverted)

test_original_labels = [test_label_encoder.inverse_transform([argmax(test_onehot_encoded[i, :])]).tolist()[0] for i in range(len(test_tree_labels_list))]
# print(test_original_labels)
print(Counter(test_original_labels).keys())
print(Counter(test_original_labels).values())

In [ ]:
## Count how many different AGB values we have among all the trees sampled. Note that I use Counter() because I assumed that there were not many different values
## We can see that although it is a regression problem, there are NOT many AGB values

## For TRAINING
train_tree_agb_values_list = [tensor_agb_value.item() for tensor_agb_value in train_tree_agb_values]
print(Counter(train_tree_agb_values_list).keys())
print(Counter(train_tree_agb_values_list).values())

print(len(Counter(train_tree_agb_values_list).values()))

## For VALIDATION
val_tree_agb_values_list = [tensor_agb_value.item() for tensor_agb_value in val_tree_agb_values]
print(Counter(val_tree_agb_values_list).keys())
print(Counter(val_tree_agb_values_list).values())

print(len(Counter(val_tree_agb_values_list).values()))

## For TESTING
test_tree_agb_values_list = [tensor_agb_value.item() for tensor_agb_value in test_tree_agb_values]
print(Counter(test_tree_agb_values_list).keys())
print(Counter(test_tree_agb_values_list).values())

print(len(Counter(test_tree_agb_values_list).values()))

### **Summary of the data (train, validation, test)**

In [ ]:
# For TRAINING
print("Total number of trees for TRAINING: ", len(train_tree_images))
print(train_tree_images[0])
print(train_tree_images[0].shape)

# Check all the information we have for one sample
print(train_tree_images[0])
print(train_labels_list[0])   # recall: OHE
print(train_tree_agb_values[0])

print(train_tree_images[0].shape)
print(train_labels_list[0])
print("A OHE label shape for TRAINING: ", train_labels_list[0].shape)


## For VALIDATION
print("Total number of trees for VALIDATION: ", len(val_tree_images))
print(val_tree_images[0])
print(val_tree_images[0].shape)

# Check all the information we have for one sample
print(val_tree_images[0])
print(val_labels_list[0])   # recall: OHE
print(val_tree_agb_values[0])

print(val_tree_images[0].shape)
print(val_labels_list[0])
print("A OHE label shape for VALIDATION: ", val_labels_list[0].shape)


## For TESTING
print("Total number of trees for TESTING: ", len(test_tree_images))
print(test_tree_images[0])
print(test_tree_images[0].shape)

# Check all the information we have for one sample
print(test_tree_images[0])
print(test_labels_list[0])   # recall: OHE
print(test_tree_agb_values[0])

print(test_tree_images[0].shape)
print(test_labels_list[0])
print("A OHE label shape for TESTING: ", test_labels_list[0].shape)

In [ ]:
# Just naming of variables for convenience
train_tree_labels = train_labels_list
val_tree_labels = val_labels_list
test_tree_labels = test_labels_list

In [ ]:
# Sanity check to see if the tree image tensors are all NOT empty

for i in range(len(train_tree_images)):
    flat_tensor = train_tree_images[i].flatten()
    if flat_tensor.all():
        print("All values in the tensor are 0...")

### **Plot some samples to see the tree images that we will feed to the model**

In [ ]:
print(train_tree_images[0].permute(1, 2, 0).shape)
plt.imshow(train_tree_images[0].permute(1, 2, 0))

In [ ]:
plt.imshow(train_tree_images[1].permute(1, 2, 0))

In [ ]:
plt.imshow(train_tree_images[2].permute(1, 2, 0))

In [ ]:
plt.imshow(val_tree_images[0].permute(1, 2, 0))

In [ ]:
plt.imshow(val_tree_images[1].permute(1, 2, 0))

In [ ]:
plt.imshow(val_tree_images[2].permute(1, 2, 0))

In [ ]:
plt.imshow(test_tree_images[0].permute(1, 2, 0))

In [ ]:
plt.imshow(test_tree_images[1].permute(1, 2, 0))

In [ ]:
plt.imshow(test_tree_images[2].permute(1, 2, 0))

### **Prepare data to train and evaluate the model**

In [ ]:
from torch._C import dtype

# Create a PyTorch class MyDataset in order to obtain a whole dataset with X (images) AND Y data (agb values) (and X features inputs in addition to the images input)
class MyDataset(Dataset):
    def __init__(self, X_data, Y_data, X_train_features, transform=None):

        # the arguments (images and targets and additional features) should all be a list of tensors

        self.tree_images = X_data
        self.tree_abg_values = Y_data
        self.tree_groups = X_train_features

        self.transform = transform

    def __len__(self):
        return len(self.tree_abg_values)

    def __getitem__(self, idx):
        
        # print(type(self.tree_images[idx]))
        # print(type(self.tree_abg_values[idx]))
        # print(type(self.tree_groups[idx]))

        # self.tree_images[idx] = self.tree_images[idx].to(torch.float)
        # self.tree_abg_values[idx] = self.tree_abg_values[idx].astype(dtype=np.float32)
        self.tree_groups[idx] = self.tree_groups[idx].to(torch.float)

        # print(self.tree_images[idx].type())
        # print(type(self.tree_abg_values[idx]))
        # print(self.tree_groups[idx].type())
        
        sample = self.tree_images[idx], self.tree_abg_values[idx], self.tree_groups[idx]

        # print(self.tree_images[idx])
        # print(self.tree_abg_values[idx])
        # print(self.tree_groups[idx])
        
        if self.transform:
            sample = self.transform(self.tree_images[idx]), self.tree_abg_values[idx], self.tree_groups[idx]

        return sample

In [ ]:
# If using the folder data (as opposed to the torchgeo dataset)

X_train_images, Y_train, X_train_features = train_tree_images, train_tree_agb_values, train_tree_labels
X_val_images, Y_val, X_val_features = val_tree_images, val_tree_agb_values, val_tree_labels, 
X_test_images, Y_test, X_test_features = test_tree_images, test_tree_agb_values, test_tree_labels

del train_tree_images
del train_tree_labels
del train_tree_agb_values
del val_tree_images
del val_tree_labels
del val_tree_agb_values
del test_tree_images
del test_tree_labels
del test_tree_agb_values

my_transform = transforms.Compose([
        transforms.Resize([224,224]), # resize the image as the ResNet18 needs 224 x 244 as input dimensions
        # transforms.RandomHorizontalFlip(),
        # transforms.ToTensor(),  # not needed as we already have a Tensor (e.g., not a ndarray)
        # transforms.Normalize(mean=(0.25,0.25,0.25), std=(0.14,0.14,0.14))
    ])

train_dataset = MyDataset(X_train_images, Y_train, X_train_features, transform=my_transform)
val_dataset = MyDataset(X_val_images, Y_val, X_val_features, transform=my_transform)
test_dataset = MyDataset(X_test_images, Y_test, X_test_features, transform=my_transform)

In [ ]:
del X_train_images
del Y_train
del X_train_features
del X_val_images
del Y_val
del X_val_features
del X_test_images
del Y_test
del X_test_features

import gc
gc.collect()

# **Model**

### **Define the model**

In [ ]:
import torchvision.models as models

## Define a simple CNN baseline model
class BaselineCNN(nn.Module):
    def __init__(self, in_channels: int, out_features: int):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=in_channels,              
            out_channels=8,            
            kernel_size=3,              
            stride=1,                   
            padding=2,                  
        )

        self.conv2 = nn.Conv2d(
            in_channels=8,              
            out_channels=16,            
            kernel_size=5,              
            stride=1,                   
            padding=2,                  
        )

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(50176, 100)  
        self.fc2 = nn.Linear(100, out_features)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))        
        x = x.view(x.size(0), -1)   # flatten the output of the convolution to (batch_size, ...)   
        x = self.relu(self.fc1(x))
        output = self.fc2(x)
    
        return output


## Define the fine-tuned model
class FineTuneModel(nn.Module):
    def __init__(self, out_features: int):
        super().__init__()

        # ResNet18
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)     # weights=ResNet18_Weights.DEFAULT or weights=ResNet18_Weights.IMAGENET1K_V1
        # ResNet50
        # self.resnet = models.resnet50(weights="IMAGENET1K_V2")     # weights=ResNet50_Weights.DEFAULT or weights=ResNet50_Weights.IMAGENET1K_V2

        # Freeze the weights of all the layers of the ResNet
        # for param in self.resnet18.parameters():
        #     param.requires_grad = False     
        
        # Change the last layer of ResNet18 to get an embedding layer to be used later with the other input which together will be mapper to a single value as we want to perform regression
        embedd_size = 256   # embedding layer size of the ResNet18 output instead of its usual classification layer; arbitrary
        self.resnet.fc = nn.Linear(512, embedd_size)  # 512 for ResNet18; by default, a newly created/modified layer gets requires_grad=True
        # self.resnet.fc = nn.Linear(2048, embedd_size)  # 2048 for ResNet50; by default, a newly created/modified layer gets requires_grad=True
        
        categorical_feature_embedd_size = 2     # e.g., 6 if we have 6 possible group labels (hence OHE vector of size 6), or 2 if we only considered the OHE banana/non-banana
        combined_embedd_size = embedd_size + categorical_feature_embedd_size  

        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(combined_embedd_size, 128)  # 128 is an arbitrary choice 
        self.final_layer = nn.Linear(128, out_features)

    def forward(self, x_image, x_group_feature_embedded):
        # print(x.shape)

        # ResNet (encoder) for the image input
        x_image_output = self.resnet(x_image)

        # Concatenate the output of the ResNet embedding layer with the OHE embedding of the categorical feature (i.e., tree group label) 
        combined_input = torch.concat([x_image_output, x_group_feature_embedded], dim=1)

        # Simple FeedForward part to get the regression output from the combined inputs (i.e., image and (OHE) categorical feature)
        output = self.relu(self.fc1(combined_input))
        output = self.final_layer(output)
    
        return output


### **Instantiate the model**

In [ ]:
## Create the model

## Baseline CNN
# self.model = BaselineCNN(in_channels=3, out_features=1)

## To directly predict with ResNet18 only
# model = models.resnet18(pretrained=True)     # weights=ResNet18_Weights.DEFAULT or weights=ResNet18_Weights.IMAGENET1K_V1
# for param in model.parameters():
    # param.requires_grad = False     # to freeze the weights of all the layers
# model.fc = nn.Linear(512, 1)    # change the last layer to a fully connected one so that the output is single value for regression; the weights have gradient set to True by default, so we keep them as such since we want to train the last layer only!

## FineTune ResNet18
model = FineTuneModel(out_features=1)   # single target regression problem, hence out_features=1

print(model)
print("\nThe number of learnable parameters in the model is: ", sum([p.numel() for p in model.parameters() if p.requires_grad]), "\n\n")

# **Experiment**

In [ ]:
if torch.cuda.is_available():
    DEVICE = 'cuda:0'
else:
    DEVICE = 'cpu'
print("Running on: ", DEVICE)

In [ ]:
class Framework(object):
    """
    General framework for experiments.
    
    """
    def __init__(self, model, train_loader, val_loader, criterion, optimizer, batch_size, num_epochs, learning_rate, *args, **kwargs):

        self.print_interval = 10    # number of batches to process before displaying the updated metrics during training

    # For training
    def train(self):
        raise NotImplementedError()

    # For evaluation
    def predict(self, eval_loader):
        estimations_batches = []
        targets_batches = []

        self.model.eval()

        for (batch_x, batch_y, batch_group_feature_x) in tqdm(eval_loader):
            
            # estimate/predict the target values
            estimated_targets = self.model(batch_x.to(DEVICE), batch_group_feature_x.to(DEVICE))
            estimations_batches.append(estimated_targets.cpu().detach().numpy())
            
            # store the associated targets batch 
            targets_batches.append(batch_y)

        output = np.concatenate(estimations_batches, axis=0)
        targets = np.concatenate(targets_batches, axis=0)
        
        return output, targets


In [ ]:
class ModelTrainer(Framework):
    """
    The Trainer for the model inheriting the general training-evaluation framework.
    
    """
    def __init__(self, model, train_loader, val_loader, criterion, optimizer, batch_size, num_epochs, learning_rate, *args, **kwargs):
        super().__init__(model, train_loader, val_loader, criterion, optimizer, batch_size, num_epochs, learning_rate, *args, **kwargs)

        # === SETUP ===
        self.model = model
        self.train_loader = train_loader
        # self.val_loader = val_loader

        # Hyperparameters and general parameters
        self.num_epochs = num_epochs # 30
        # self.learning_rate = learning_rate # 1e-4
        # self.weight_decay_tuned = 5e-3
        
        # params = filter(lambda p: p.requires_grad, self.model.parameters())     # filter out the parameters to NOT be fine-tuned; allows to specify to the optimizer the weights to update
        self.optimizer = optimizer # torch.optim.Adam(params, lr=self.learning_rate)   # self.model.parameters()
        # self.lrs = torch.optim.lr_scheduler.StepLR(self.optimizer, 10, 0.7, verbose=True)   # for learning rate scheduling
        self.criterion = criterion # nn.MSELoss()

        # ===

    def train(self):
        self.model.train()
        progress_bar = trange(self.num_epochs)
        for _ in progress_bar:
            for batch_idx, (batch_images_x, batch_y, batch_group_feature_x) in enumerate(self.train_loader):

                self.model.zero_grad()

                # Set data to gpu if possible
                batch_images_x = batch_images_x.to(DEVICE)
                batch_y = batch_y.to(DEVICE)
                batch_group_feature_x = batch_group_feature_x.to(DEVICE)

                # Perform forward pass
                estimated_target = self.model(batch_images_x, batch_group_feature_x)

                loss = self.criterion(estimated_target, batch_y.unsqueeze(1))   # unsqueeze the batch_y to avoid a warning that could lead to stacking errors; here we had estimated_target of size 64 and batch_y of size 64 x 1
                
                wandb.log({"MSE loss (each batch) ": loss.item()})
                batch_rmse = mean_squared_error(batch_y.cpu(), estimated_target.cpu().detach().numpy(), squared=False)
                wandb.log({f"RMSE (each batch) ": batch_rmse})

                # Backpropagate to get the gradients
                loss.backward()

                self.optimizer.step()

                # Update progress bar for training every self.print_interval batches
                if batch_idx % self.print_interval == 0:
                    # output = self.model(batch_images_x, batch_group_feature_x)
                    output = estimated_target
                    current_rmse = mean_squared_error(batch_y.cpu(), output.cpu().detach().numpy(), squared=False)
                    wandb.log({f"RMSE (of a batch) every {self.print_interval} batches": batch_rmse})
                    progress_bar.set_postfix(loss=loss.item(), current_rmse=batch_rmse)

In [ ]:
def evaluate(model_trainer:Framework, eval_loader):
    """
    :param model: Trained model to be evaluated
    :param eval_loader: The validation loader
    """

    # Get the predictions for the whole eval_loader
    output, targets = model_trainer.predict(eval_loader)

    output = output.flatten()   # to have exactly same size as targets
    # targets = np.expand_dims(targets, 1)    # to have exactly same size as output if we did not change the size of targets
    
    print("\nResult with biggest difference: ", np.max(np.abs(output-targets)))
    # print("\n\nOnly print the first 20 predictions and targets:\n")
    # print("Predictions: \n", output[:20])
    # print("Targets: \n", targets[:20])
    # print("Difference between output and targets: \n", output[:20]-targets[:20])
    # print("Output shape: ", output.shape)
    # print("Targets shape: ", targets.shape)

    # Calculate evaluation metrics
    # error_from_scratch = np.sqrt(((output-targets)**2).mean())  # compute RMSE in pure numpy
    # print(error_from_scratch)  
    error = mean_squared_error(targets, output, squared=False)      # with squared=False we get the RMSE (as in the paper)

    print("--- Evaluation ---")
    print(f'Root Mean Squared Error: {error:.5f}')

In [ ]:
def run_solution(model, train_loader, val_loader, criterion, optimizer, batch_size=32, num_epochs=20, learning_rate=1e-4):
    if val_loader is None:
        val_loader = train_loader

    trainer = ModelTrainer(model, train_loader, val_loader, criterion, optimizer, batch_size=batch_size, num_epochs=num_epochs, learning_rate=learning_rate)
    trainer.train()

    if val_loader is None:
        print('Evaluating model on TRAINING data (and NOT on validation data) !')

    # evaluate(trainer, eval_loader=val_loader)

    return trainer

In [ ]:
# Define the main hyperparameters

BATCH_SIZE = 64
LR = 1e-4
N_EPOCHS = 30  # before 2000

params = filter(lambda p: p.requires_grad, model.parameters())   # filter out the parameters to NOT be fine-tuned; allows to specify to the optimizer the weights to update
OPTIMIZER = torch.optim.Adam(params, lr=LR)
CRITERION = nn.MSELoss()      


In [ ]:
# Create and save the three dataloaders

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, num_workers=2)     # use dataset_train to check if learning for now (????)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, num_workers=2)     # use dataset_train to check if learning for now (????)

# del train_dataset
# del val_dataset
# del test_dataset

# Save the newly created dataloaders
!mkdir dataloaders
torch.save(train_loader, './dataloaders/train_dataloader.pt')
torch.save(val_loader, './dataloaders/val_dataloader.pt')
torch.save(test_loader, './dataloaders/test_dataloader.pt')

In [ ]:
# Zip the dataloader folder in which we saved the created dataloaders

# zip the folder
# !zip -r dataloaders.zip ./dataloaders

# save the zipped folder in my Gdrive
# !cp dataloaders.zip '/content/drive/MyDrive/ETHZ/AI4Good/'
# !ls -lt '/content/drive/MyDrive/ETHZ/AI4Good/' 

# from google.colab import files
# files.download('./dataloaders.zip')

In [ ]:
del train_loader
del val_loader
del test_loader

In [ ]:
# Once dataloaders are available in the working directory, I can just run the cells from here until the end to run a full experiment
# Load the dataloader
train_loader = torch.load('./dataloaders/train_dataloader.pt')
val_loader = torch.load('./dataloaders/val_dataloader.pt')
test_loader = torch.load('./dataloaders/test_dataloader.pt')

**WandB settings**

In [ ]:
wandb.login()

%env "WANDB_NOTEBOOK_NAME" "AI4Good-ReforesTree-Main"

# wandb.init()

# wandb.init(project="AI4Good-ReforesTree", 
#            entity="Klim",
#            config={
#                "epochs": N_EPOCHS,
#                "batch_size": BATCH_SIZE,
#                "learning_rate": LR,
#                "optimizer": OPTIMIZER,
#                "criterion": CRITERION,
#                "device": DEVICE,
#                "dataset": "reforestree"
#            })

wandb.init(config={
               "epochs": N_EPOCHS,
               "batch_size": BATCH_SIZE,
               "learning_rate": LR,
               "optimizer": OPTIMIZER,
               "criterion": CRITERION,
               "device": DEVICE,
               "dataset": "reforestree"
           })

# set model to wandb
wandb.watch(model, log_freq=100)

In [ ]:
# If cell outputs wandb.run, we see the live graphs
# wandb.run


**Run experiment**

In [ ]:
# %%wandb

# Set the model to GPU if available
model = model.to(DEVICE)

print("Running on GPU: ", torch.cuda.is_available(), "\n")
# Run actual solution and returned the trained model within the trainer 
model_trainer = run_solution(model, train_loader, val_loader, CRITERION, OPTIMIZER, BATCH_SIZE, N_EPOCHS, LR)

### **Display data images with values in WandB**

In [ ]:
## Display a few samples of the dataloaders in wandb
nb_of_batches_to_display_from = 3
nb_of_samples_per_batch_to_display = 2
data = []

iter_train_loader = iter(train_loader)
iter_val_loader = iter(val_loader)
iter_test_loader = iter(test_loader)

for i in range(nb_of_batches_to_display_from):   # iterate over batches

    if i % 3 == 0:
        tree_image_batch, tree_agb_value_batch, tree_group_label_batch = next(iter_train_loader) 
    elif i % 3 == 1:
        tree_image_batch, tree_agb_value_batch, tree_group_label_batch = next(iter_val_loader) 
    elif i % 3 == 2:
        tree_image_batch, tree_agb_value_batch, tree_group_label_batch = next(iter_test_loader) 

    for j in range(nb_of_samples_per_batch_to_display):   # iterate over samples in batch i

        tree_image = tree_image_batch[j]
        tree_agb_value = tree_agb_value_batch[j]
        tree_group_label = tree_group_label_batch[j]

        # print(tree_image.shape) 
        # print(tree_agb_value.shape) 
        # print(tree_group_label.shape) 

        image = wandb.Image(tree_image, caption=f"Tree crown image in batch {i} sample {j}")  # convert Tensor image to wandb image format
        
        tree_group_label = torch.argmax(tree_group_label)   # since the tree group labels are OHE, get the label as 0=non-banana and 1=banana

        data.append([image, tree_group_label, tree_agb_value])

images_table = wandb.Table(columns=["image", "group label", "target_gt"], data=data)

# images_table.add_column("id", ids)
# images_table.add_column("image", images)
# images_table.add_column("group label", group_labels)
# images_table.add_column("target_prediction", agb_values)

# Log the Table to W&B
wandb.log({"some ReforesTree data in the loaders": images_table})

### **Perform inference on the test set to evaluate the final performance of the trained model**

In [ ]:
test_output, test_targets = model_trainer.predict(test_loader)

test_output = test_output.flatten()   # to have exactly same size as targets
# targets = np.expand_dims(targets, 1)    # to have exactly same size as output

print("\Prediction with biggest difference to ground truth: ", np.max(np.abs(test_output-test_targets)))
print("\n\nOnly print the predictions and targets:\n")
print("Predictions: \n", test_output)
print("Targets: \n", test_targets)
print("Difference between output and targets: \n", test_output-test_targets)
print("Output shape: ", test_output.shape)
print("Targets shape: ", test_targets.shape)

# Calculate evaluation metrics
# error_from_scratch = np.sqrt(((output-targets)**2).mean())  # compute RMSE in pure numpy
# print(error_from_scratch)  
test_error = mean_squared_error(test_targets, test_output, squared=False)      # with squared=False we get the RMSE (as in the paper)

print("--- TESTING EVALUATION ---")
print(f'Root Mean Squared Error (on test set): {test_error:.5f}')

### **Display TEST data images with values in WandB**

In [ ]:
## Display a few samples of the testloader as well as predictions in wandb
nb_of_batches_to_display_from = 3
nb_of_samples_per_batch_to_display = 2
data = []

iter_test_loader = iter(test_loader)
for i in range(nb_of_batches_to_display_from):  # iterate over batches
    
    tree_image_batch, tree_agb_value_batch, tree_group_label_batch = next(iter_test_loader) 

    for j in range(nb_of_samples_per_batch_to_display):   # iterate over samples in batch i


        tree_image = tree_image_batch[j]
        tree_agb_value = tree_agb_value_batch[j]
        tree_group_label = tree_group_label_batch[j]

        # print(tree_image.shape) 
        # print(tree_agb_value.shape) 
        # print(tree_group_label.shape) 

        image = wandb.Image(tree_image, caption=f"Tree crown image in batch {i} sample {j}")      # convert Tensor image to wandb image format
        
        tree_group_label = torch.argmax(tree_group_label)   # since the tree group labels are OHE, get the label as 0=non-banana and 1=banana

        data.append([image, tree_group_label, tree_agb_value, test_output[i*(BATCH_SIZE - 1) + j]])   # Correct how I get the predictions related to the gt agb value??? We get the first element of each batch i

images_table = wandb.Table(columns=["image", "group label", "target_gt", "target_prediction"], data=data)

# Log the Table to W&B
wandb.log({"some ReforesTree data in the testloader + predictions": images_table})

In [ ]:
# experiment_run.finish()

In [ ]:
torch.save(model_trainer.model.state_dict(), '../ckpt/trained_model.ckpt')